In [1]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
import pandas as pd


c:\Users\jorgen\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda', index=0)

In [4]:

data = load_dataset("navjordj/SNL_summarization", split="validation").shuffle().select(range(10))
metric = evaluate.load("rouge", use_stemming=True)



Using custom data configuration navjordj--SNL_summarization-d89e75929608302f
Found cached dataset parquet (C:/Users/jorgen/.cache/huggingface/datasets/navjordj___parquet/navjordj--SNL_summarization-d89e75929608302f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
model = T5ForConditionalGeneration.from_pretrained("navjordj/t5-base-snl")
tokenizer = AutoTokenizer.from_pretrained("navjordj/t5-base-snl")


In [5]:
greedy_config = {
}

beam_config = {
    "num_beams": 6,
    "early_stopping": True
}

beam_no_stopping_config = {
    "num_beams": 6,
    "early_stopping": False
}

beam_rep_penalty_1_2_config = {
    "num_beams": 6,
    "repetition_penalty": 1.2
}

beam_rep_penalty_2_config = {
    "num_beams": 6,
    "repetition_penalty": 2.0
}

beam_no_ngram_2 = {
    "num_beams": 6,
    "no_repeat_ngram_size": 2, 
    "early_stopping": True
}

beam_no_ngram_4 = {
    "num_beams": 6,
    "no_repeat_ngram_size": 4, 
    "early_stopping": True
}

beam_no_ngram_12 = {
    "num_beams": 6,
    "no_repeat_ngram_size": 12, 
    "early_stopping": True
}

sample_config = {
    "do_sample": True,
    "top_k": 0
}

sample_temperature_config = {
    "do_sample": True,
    "top_k": 0,
    "temperature": 0.7
}

top_k_config = {
    "do_sample": True,
    "top_k": 50,
}

top_p_config = {
    "do_sample": True,
    "top_p": 0.92, 
    "top_k": 0
}

configs = [greedy_config, beam_config, beam_no_stopping_config, beam_rep_penalty_1_2_config, beam_rep_penalty_2_config, beam_no_ngram_2, beam_no_ngram_4, sample_config, sample_temperature_config, top_k_config, top_p_config]


In [6]:
task_evaluator = evaluator("summarization")

results_df = pd.DataFrame(columns=["config", 'rouge1','rouge2', 'rougeL', 'rougeLsum', 'total_time_in_seconds', 'samples_per_second'])

for config in configs[:1]:
    print(config)
    pipe = pipeline("summarization", model=model, tokenizer=tokenizer, batch_size=1, device=device, **config)

    results = task_evaluator.compute(
        model_or_pipeline=pipe,
        data=data,
        input_column="article",
        label_column="ingress",
        metric=metric
    )
    print(results)

    results_df = results_df.append({"config": config, **results}, ignore_index=True)

results_df.to_csv("hyperparam_search_results.csv")


{}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'rouge1': 0.3353283203455176, 'rouge2': 0.17750911367429723, 'rougeL': 0.2964294395377383, 'rougeLsum': 0.29735717702646003, 'total_time_in_seconds': 9.976816200000002, 'samples_per_second': 1.0023237673758085, 'latency_in_seconds': 0.9976816200000002}


C:\Users\jorgen\AppData\Local\Temp\ipykernel_12720\3407022739.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"config": config, **results}, ignore_index=True)


In [15]:
pipe = pipeline("summarization", model=model, tokenizer=tokenizer, batch_size=1, device=device, **config, max_length=120)

pipe(data[0]["article"])




[{'summary_text': 'Broderi er en teknikk som i særlig grad omfatter brodering av ullgarn, ullgarn og ullgarn.'}]